In [ ]:
import torch
import plotly.graph_objects as go
from plotly.colors import qualitative

# -------------------- 构建动画图 --------------------
# 加载保存的 pt 文件
map_data = torch.load("./map_data.pt")
map_mask_data = torch.load("./map_mask_data.pt")
agent_position_data = torch.load("./agent_position_data.pt")
agent_mask_data = torch.load("./agent_mask_data.pt")

# 定义地图车道颜色组，每组包含中心线、左边界和右边界的颜色
color_pairs = [
    {"center": "#2C3E50", "left": "#1f77b4", "right": "#d62728"},  # 组0：中心线带蓝色调
    {"center": "#34495E", "left": "#17becf", "right": "#ff7f0e"},  # 组1：中心线带蓝灰调
    {"center": "#8E44AD", "left": "#0074D9", "right": "#FF4136"},  # 组2：中心线带紫色调
    {"center": "#16A085", "left": "#2E86C1", "right": "#E74C3C"},  # 组3：中心线带青绿色调
    {"center": "#27AE60", "left": "#2980B9", "right": "#E67E22"},  # 组4：中心线带绿色调
    {"center": "#D35400", "left": "#3498DB", "right": "#F39C12"},  # 组5：中心线带橙色调
    {"center": "#C0392B", "left": "#5DADE2", "right": "#E74C3C"},  # 组6：中心线带红色调
]

# 计算所有帧中 agent 数量的最大值
max_A = max([agent_position_item.shape[0] for agent_position_item in agent_position_data])
# 使用 Plotly 提供的定性配色方案作为 agent 的颜色池（不足则循环使用）
agent_colors_pool = qualitative.Plotly  
agent_colors = [agent_colors_pool[i % len(agent_colors_pool)] for i in range(max_A)]

frames = []
n_frames = len(map_data)

# 为保证每一帧中 trace 数量和顺序一致：
# 地图车道部分：按车道顺序，每个车道固定添加中心线、左边界、右边界；
# agent 部分：始终循环 0 ~ max_A，即使当前帧该 agent 无数据也添加空 trace，
# 这样不同帧中相同的 agent（A 维度相同位置）始终使用相同颜色。
for frame_idx, (map_item, map_mask_item, agent_position_item, agent_mask_item) in enumerate(
        zip(map_data, map_mask_data, agent_position_data, agent_mask_data)):
    
    traces = []
    # 地图数据：假设 map_item.shape 为 [D1, D2, P, 2]，其中 D1 为车道数量
    D1, _, _, _ = map_item.shape
    for lane_idx in range(D1):
        color_set = color_pairs[lane_idx % len(color_pairs)]
        lane_mask = map_mask_item[lane_idx]  # 布尔 mask, shape: [P]
        lines = map_item[lane_idx]           # shape: [D2, P, 2]
        center_line = lines[0]
        left_bound = lines[1]
        right_bound = lines[2]
        # 筛选有效点
        center_valid = center_line[lane_mask]
        left_valid = left_bound[lane_mask]
        right_valid = right_bound[lane_mask]
        
        # 始终添加三个 trace（数据为空时 x, y 为 []）
        traces.append(go.Scatter(
            x=center_valid[:, 0].tolist() if center_valid.shape[0] > 0 else [],
            y=center_valid[:, 1].tolist() if center_valid.shape[0] > 0 else [],
            mode='lines',
            name=f"Lane {lane_idx} 中心线",
            line=dict(color=color_set["center"], dash="dot")
        ))
        traces.append(go.Scatter(
            x=left_valid[:, 0].tolist() if left_valid.shape[0] > 0 else [],
            y=left_valid[:, 1].tolist() if left_valid.shape[0] > 0 else [],
            mode='lines',
            name=f"Lane {lane_idx} 左边界",
            line=dict(color=color_set["left"])
        ))
        traces.append(go.Scatter(
            x=right_valid[:, 0].tolist() if right_valid.shape[0] > 0 else [],
            y=right_valid[:, 1].tolist() if right_valid.shape[0] > 0 else [],
            mode='lines',
            name=f"Lane {lane_idx} 右边界",
            line=dict(color=color_set["right"])
        ))
    
    # Agent 数据：agent_position_item.shape 为 [A, P, 2]，agent_mask_item.shape 为 [A, P]
    A = agent_position_item.shape[0]
    select_agent_id = 2
    
    for agent_idx in range(max_A):
        if agent_idx < A:# and agent_idx==select_agent_id:
            agent_line = agent_position_item[agent_idx]  # shape: [P, 2]
            agent_mask = agent_mask_item[agent_idx]        # shape: [P]
            valid_points = agent_line[agent_mask]
        else:
            valid_points = torch.empty((0, 2))
        
        traces.append(go.Scatter(
            x=valid_points[:, 0].tolist() if valid_points.shape[0] > 0 else [],
            y=valid_points[:, 1].tolist() if valid_points.shape[0] > 0 else [],
            mode='lines+markers',
            name=f"Agent {agent_idx}",
            line=dict(color=agent_colors[agent_idx])
        ))
    
    frames.append(dict(data=traces, name=str(frame_idx)))
    
# 初始帧数据采用第 0 帧（所有帧中 trace 数量和顺序均一致）
initial_data = frames[0]['data'] if frames else []

# 根据第一帧数据，确定车道 trace 数量（假设每帧车道数量一致）
initial_D1 = map_data[0].shape[0]
lane_trace_count = initial_D1 * 3   # 每个车道 3 个 trace
agent_trace_count = max_A           # agent 部分 trace 数量

# 计算各组 trace 索引
lane_indices = list(range(lane_trace_count))
agent_indices = list(range(lane_trace_count, lane_trace_count + agent_trace_count))

# 构造 Figure 对象，添加播放/暂停按钮、帧滑动条，以及两个下拉菜单用于独立控制车道和 agent 的显示
fig = go.Figure(
    data=initial_data,
    layout=go.Layout(
        title="Map & Agent Data (Consistent Traces)",
        xaxis=dict(title="X"),
        yaxis=dict(title="Y", scaleanchor="x", scaleratio=1),
        template="plotly_white",
        width=800,
        height=800,
        uirevision="constant",  # 保持轴状态不刷新
        updatemenus=[
            # 播放/暂停按钮（放在图下中间偏左）
            {
                "type": "buttons",
                "showactive": False,
                "x": 0,
                "y": -0.3,
                "xanchor": "left",
                "yanchor": "top",
                "pad": {"t": 10, "r": 10},
                "buttons": [
                    {
                        "label": "播放",
                        "method": "animate",
                        "args": [None, {"frame": {"duration": 0, "redraw": True},
                                        "fromcurrent": True, "transition": {"duration": 0}}]
                    },
                    {
                        "label": "暂停",
                        "method": "animate",
                        "args": [[None], {"frame": {"duration": 0, "redraw": False},
                                          "mode": "immediate",
                                          "transition": {"duration": 0}}]
                    }
                ]
            },
            # 下拉菜单：独立控制车道显示（放在图下左侧）
            {
                "type": "dropdown",
                "direction": "down",
                "showactive": True,
                "x": 0,
                "y": -0.1,
                "xanchor": "left",
                "yanchor": "top",
                "pad": {"t": 10, "r": 10},
                "buttons": [
                    {
                        "label": "显示车道",
                        "method": "restyle",
                        "args": [{"visible": True}, lane_indices]
                    },
                    {
                        "label": "隐藏车道",
                        "method": "restyle",
                        "args": [{"visible": False}, lane_indices]
                    }
                ]
            },
            # 下拉菜单：独立控制 agent 显示（放在图下右侧）
            {
                "type": "dropdown",
                "direction": "down",
                "showactive": True,
                "x": 1,
                "y": -0.1,
                "xanchor": "right",
                "yanchor": "top",
                "pad": {"t": 10, "r": 10},
                "buttons": [
                    {
                        "label": "显示 Agent",
                        "method": "restyle",
                        "args": [{"visible": True}, agent_indices]
                    },
                    {
                        "label": "隐藏 Agent",
                        "method": "restyle",
                        "args": [{"visible": False}, agent_indices]
                    }
                ]
            }
        ],
        # 滑动条放在图下中间
        sliders=[{
            "active": 0,
            "x": 0.5,
            "y": -0.5,
            "xanchor": "center",
            "yanchor": "top",
            "currentvalue": {
                "font": {"size": 20},
                "prefix": "Frame: ",
                "visible": True,
                "xanchor": "right"
            },
            "pad": {"b": 10, "t": 50},
            "len": 0.9,
            "steps": [{
                "args": [[str(k)],
                         {"frame": {"duration": 0, "redraw": True},
                          "mode": "immediate",
                          "transition": {"duration": 0}}],
                "label": str(k),
                "method": "animate"
            } for k in range(n_frames)]
        }]
    ),
    frames=frames
)

fig.show()

# -------------------- 绘制统计曲线 --------------------
# 统计各帧车道数量（map_item 第一维度）和 agent 数量（agent_position_item 第一维度）
frame_indices = []
lane_counts = []
agent_counts = []
for frame_idx, (map_item, agent_position_item) in enumerate(zip(map_data, agent_position_data)):
    frame_indices.append(frame_idx)
    lane_counts.append(map_item.shape[0])
    agent_counts.append(agent_position_item.shape[0])

# 使用 Plotly 绘制统计曲线
fig_counts = go.Figure()
fig_counts.add_trace(go.Scatter(
    x=frame_indices,
    y=lane_counts,
    mode='lines+markers',
    name='车道数量'
))
fig_counts.add_trace(go.Scatter(
    x=frame_indices,
    y=agent_counts,
    mode='lines+markers',
    name='Agent 数量'
))
fig_counts.update_layout(
    title="各帧车道数量与 Agent 数量统计",
    xaxis_title="帧序号",
    yaxis_title="数量",
    template="plotly_white"
)
fig_counts.show()
